In [1]:
#导入tkinter模块的所有内容
from tkinter import*  
#爬虫需要用的包
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from urllib import request
import requests
import os
import re

In [27]:
#Frame 布局
root=Tk();root.title("ArchDaily")  #窗口标题
f1=Frame(root);f1.pack()  #界面分为三个Frame
f2=Frame(root);f2.pack()
f3=Frame(root);f3.pack()
def Print():
    global var#全局变量，用来储存客户需要的URL链接
    var = entry1.get()
    print(var)#试打印
    global Num#全局变量，用来储存客户需要的URL链接
    Num = entry2.get()
    print(Num)#试打印
    return True

Label(f1,text="网站").pack(side=LEFT)  #标签放置在f1中，左停靠
entry1=Entry(f1, validate='focusout')
entry1.pack(side=LEFT) #单行文本框放置在f1中，左停靠

Label(f2,text="页数").pack(side=LEFT)  #标签放置在f1中，左停靠
entry2=Entry(f2, validate='focusout',validatecommand=Page_Number)
entry2.pack(side=LEFT) #单行文本框放置在f1中，左停靠

button1=Button(f3,text="确定",command=Print)
button1.pack(side=RIGHT)#取消按钮放置在f3中，右停靠
root.mainloop()

https://www.archdaily.cn/cn/search/projects/categories/wen-hua-jian-zhu/country/zhong-guo
4


In [29]:
header = {
'Cookie':'__io_lv=1533872652313; __io=043eef8f9.48f07b97a_1533872652322; _io_un=; _io_un=; _io_un=10; _ga=GA1.2.309009163.1533872652; _gid=GA1.2.1093896309.1533872652; __gads=ID=093aadeb232e8f25:T=1533872652:S=ALNI_MZFvuvlvSVAOMeD6PoItbB-PGIT8Q; ad-consented_at=2018-08-10T03%3A44%3A34.886Z; kenneth_invitation=2018-08-10; __io_uid_test=3; kenneth_invitation_small=2018-08-10; _io_conversion_un=; _io_s=5; _nimrod_session=RjVkTXQ1NWVzRDA5UWlnSXM2d2dRV1ljV3RkT0l1aUlUNDl2ZjNnVXI1ME44ZDdNeDNWR1FhOGxFSzl1QjY1YjRRTVBGeHRhbVUvSmlreVJvMXJlTWhZRjVSd2ZGd0xSNEhGdGI3WmtXNlpjYmdpYXJuc2l3KzVBdEJrT1YvWWc5ZFZVNXI0eXo1Ym1DZXl4OHlYb3NwOVRYbFA1SGxFNyszZVBlOFNXaFdKRzlaWGVEL3JHeWRmb0xKNnF1Q1orLS1YSzE2NTBvNkowS2h2M1F1OVdaSndBPT0%3D--cdc53ee09ebf19285452a1c8f3425194c2589356',
'Host':'www.archdaily.cn',
'Upgrade-Insecure-Requests':'1',
'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
}
page=list(range(1,int(Num),1))#输入要爬取的页数范围
for i in page:
    url1=var+'?page='+str(i)#ArchDaily网页
    html1 = requests.get(url1,headers=header).content#遍历每一页
    soup1 = BeautifulSoup(html1, "html.parser")
    #soup1#试打印

    #新建文件夹
    Folder_Name=[]
    for a in soup1.find_all('h2',class_="afd-search-list__title"):
        #print(a.string[0:a.string.find("/")-1])
        Folder_Name.append(a.string[0:a.string.find("/")-1])
        os.mkdir("IMG//%s\\"%a.string[0:a.string.find("/")-1])
    print(len(Folder_Name))

    #将当页所有项目储存在Url数组中   
    Url=[]
    for a in soup1.find_all('a',class_="afd-search-list__link"):
        #print(a['href'])
        Url.append(a['href'])
    #str('https://www.archdaily.cn')+Url[0]
    print(len(Url))

    #获得每张照片背后的Gallery信息，其实一张照哦片就是足够的
    for i in range(0,len(Url)):
        #print(str('https://www.archdaily.cn')+Url[i])

        Gallery=[]
        html1 = requests.get(str('https://www.archdaily.cn')+Url[i], headers=header).content#获取单个项目网页内容
        soup1 = BeautifulSoup(html1, "html.parser")
        for a in soup1.select('#gallery-thumbs > li > a'):
            Gallery.append(a['href'])


        html1 = requests.get(str('https://www.archdaily.cn')+Gallery[0], headers=header).content#试验第一个图片背后Gallery就OK
        soup1 = BeautifulSoup(html1, "html.parser")
        originalElem = soup1.select('#gallery-items')
        originalLinkList = []
        for item in originalElem:
            string=str(item.get('data-images'))#获得Gallery 网页的文字信息用正则表达式筛选
            #print(item.get('data-images'))
            #这个地方必须用正则表达式
        #正则表达式提取有用的URL

        pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')  # 查找数字
        result1 = pattern.findall(string)
        Img=[]#用来储存筛选Gallery后的Img链接，用来下载
        for j in range(0,len(result1)):
            if result1[j].find("large_jpg")==-1:
                continue
            Img.append(result1[j])

        #下载图片
        print(Folder_Name[i]+"开始下载")
        for j in range(0,len(Img)):
            response=requests.get(Img[j])
            img=response.content
            string ='IMG\\'+Folder_Name[i]+'\\'+str(j)+'.jpg'
            #string = 'G\\'+title[i] +'\\'+str(j)+'.jpg'
            with open(string,'wb') as f:
                f.write(img)
                f.close        
        print(Folder_Name[i]+"完成下载")




24
24
黄石市城市综合馆开始下载
黄石市城市综合馆完成下载
南方科技大学图书馆开始下载
南方科技大学图书馆完成下载
FIUFIU 画廊开始下载
FIUFIU 画廊完成下载
上海世博公园“不确定的回忆”装置开始下载
上海世博公园“不确定的回忆”装置完成下载
广州粤剧艺术博物馆开始下载
广州粤剧艺术博物馆完成下载
石仓契约博物馆开始下载
石仓契约博物馆完成下载
北京保利未来大都汇展示中心开始下载
北京保利未来大都汇展示中心完成下载
謇公湖开始下载
謇公湖完成下载
上海陇上书店开始下载
上海陇上书店完成下载
白“止间”书店 开始下载
白“止间”书店 完成下载
西安钟书阁开始下载


KeyboardInterrupt: 